In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto",  torch_dtype=torch.bfloat16,  cache_dir="/data")



In [1]:
input_text = "Help me write a prompt as a question from a concept and an attribute. \nConcept: monkey\nAttribute: appears_in_comics.\nPrompt: In one word Yes/No <mask> ?"

152

In [4]:
input_text = "In one word Yes/No, is the feature related to the concept.\nConcept: {}\Feature: {}.\n <mask>".format("monkey", "appears_in_comics")

In [4]:
input_text = "Q: Is the property [is_female] true for the concept [book]?\nA: False\nQ: Is the property [can_be_digital] true for the concept [book]?\nA: True\nIn one word True/False answer the following question:\nQ: : Is the property [{}] true for the concept [{}]?\nA: <mask>".format("monkey", "carries_a_ring")

In [5]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> False</s>
